In [ ]:
print('Hello LangChain')

In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="자바는 무엇인가요? 자세하게 설명해주세요")

In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("" \
"You are an expert in AI Expert." \
"Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

In [ ]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

### LCEL
* Prompt + LLM + outputparser를 Chain으로 연결하기

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

In [ ]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

In [ ]:
# chain 호출
try:
    result = chain2.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe같은 products가 있어."})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

### Runnable의 stream() 함수 호출

In [ ]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해주세요."})
    # 스트리밍 출력
    print(answer)

    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")

### Multi Chain
* 첫 번째 Chain의 출력이, 두 번째 Chain의 입력이 된다.
* 두 개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()


In [ ]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "Drama"})
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

### PromptTemplate 여러개 연결하기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

In [19]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 요약해서 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 요약해서 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 요약해서 한국어로 답변해 주세요.']


In [ ]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) # AI Message 타입
    print(response)

content='GPT-4 모델은 대규모 데이터셋을 기반으로 하는 딥러닝 알고리즘을 사용하여 학습되며, 자연어 처리 작업을 위해 특별히 설계된 트랜스포머 아키텍처를 활용합니다. 이 모델은 주어진 문맥을 이해하고 적절한 응답을 생성하도록 학습되며, 강화 학습 및 지도 학습을 통해 지속적으로 개선됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 30, 'total_tokens': 103, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.256568951, 'prompt_time': 0.003246002, 'completion_time': 0.154141496, 'total_time': 0.157387498}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'id': 'chatcmpl-5691ae20-c3cc-4b2f-8e2b-d7852b11d169', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--a2a2b8a2-bca7-4281-aa2a-20af43721128-0' usage_metadata={'input_tokens': 30, 'output_tokens': 73, 'total_tokens': 103, 'input_token_details': {}, 'output_token_details': {}}
content='Gemma는 컴퓨터가 자연어를 이해하고 생성할 수 있도록 설계된 언어 모델입니다. Gemma는 대규모의 텍스트 데이터를 학습하여 언어의 패턴과 구조를 파악하고, 이를 기반으로 새로운 

### ChatMessagePromptTemplate
* SystemMessagePrompt, HumanMessagePromptTemplate, AIMessagePromptTemplate를 사용

In [22]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "{topic}에 대한 예시 답입니다."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요??")

# LLM 호출
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 

딥러닝은 인공신경망을 기반으로 하는 머신러닝의 한 분야입니다. 인공신경망은 인간의 뇌를 모방한 구조로, 데이터를 처리하고 학습하는 방식입니다. 딥러닝은 이러한 인공신경망을 깊게 쌓아서 복잡한 데이터들을 처리하고 학습하는 기술입니다.

딥러닝은 데이터에서 자동으로 특징을 추출하고, 이를 통해 데이터를 분류하거나 예측하는 작업을 수행합니다. 이를 위해 딥러닝 모델은 대량의 데이터를 필요로 하며, 이를 통해 모델의 성능을 높일 수 있습니다.

딥러닝의 핵심 개념은 다음과 같습니다.

1. 인공신경망: 딥러닝은 인공신경망을 기반으로 합니다. 인공신경망은 입력층, 은닉층, 출력층으로 구성되며, 각 층은 여러 개의 노드로 구성됩니다.
2. 깊이: 딥러닝은 인공신경망을 깊게 쌓아서 복잡한 데이터들을 처리합니다. 이는 데이터에서 자동으로 특징을 추출하고, 이를 통해 데이터를 분류하거나 예측하는 작업을 수행합니다.
3. 학습: 딛러닝은 대량의 데이터를 필요로 하며, 이를 통해 모델의 성능을 높일 수 있습니다. 모델은 데이터를 학습하면서 자동으로 특징을 추출하고, 이를 통해 데이터를 분류하거나 예측하는 작업을 수행합니다.

딥러닝은 다양한 분야에서 활용되고 있습니다. 예를 들어, 이미지 인식, 음성 인식, 자연어 처리, 추천 시스템 등이 있습니다. 또한, 딥러닝은 자율 주행 자동차, 의료 진단, 금융 예측 등 다양한 분야에서 활용되고 있습니다.

딥러닝의 장점은 다음과 같습니다.

* 복잡한 데이터 처리: 딥러닝은 복잡한 데이터를 처리할 수 있습니다.
* 자동화된 특징 추출: 딥러닝은 데이터에서 자동으로 특징을 추출할 수 있습니다.
* 높은 성능: 딥러닝은 대량의 데이터를 학습하면서 높은 성능을 발휘할 수 있습니다.

하지만, 딥러닝의 단점은 다음과 같습니다.

* 대량의 데이터 필요: 딥러닝은 대량의 데이터를 필요로 합니다.
* 높은 계산 비용: 딥러닝은 높은 계산 비용이 필요합니다.
* 해석 어려움: 딥러닝 모델의 결과를 해석하기 어려울 수 있습니다.

결론적으로, 딥

### FewShotPromptTemplate
* 예시를 제공하는 프롬프트

In [31]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 태양과 가장 가까운 행성으로, 매우 작고 온도가 극심하게 변합니다.
2. **금성**: 밝고 뜨거운 행성으로, 강한 온실 효과로 표면 온도가 매우 높습니다.
3. **지구**: 생명체가 사는 유일한 행성으로, 물과 대기가 있습니다.
4. **화성**: 붉은 행성으로, 과거에는 물이 있었을 것으로 추정됩니다.
5. **목성**: 태양계에서 가장 큰 행성으로, 가스 거인입니다.
6. **토성**: 아름다운 고리를 가진 가스 거인입니다.
7. **천왕성**: 자전축이 기울어져 있어 극단적인 계절 변화를 겪습니다.
8. **해왕성**: 태양계에서 가장 먼 행성으로, 강한 바람이 불어오는 곳입니다.
